<a href="https://colab.research.google.com/github/pierredumontel/formula-one-spark-project/blob/master/Colab-notebooks/dashboards/f1_dashboards_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%run /content/drive/MyDrive/formula-one/f1-csv-ingestion.ipynb 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5836k  100 5836k    0     0  1447k      0  0:00:04  0:00:04 --:--:-- 1447k
Archive:  /content/f1db_csv.zip
replace /content/f1_datasets/circuits.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/f1_datasets/circuits.csv  
  inflating: /content/f1_datasets/constructor_results.csv  
  inflating: /content/f1_datasets/constructors.csv  
  inflating: /content/f1_datasets/constructor_standings.csv  
  inflating: /content/f1_datasets/drivers.csv  
  inflating: /content/f1_datasets/driver_standings.csv  
  inflating: /content/f1_datasets/lap_times.csv  
  inflating: /content/f1_datasets/pit_stops.csv  
  inflating: /content/f1_datasets/qualifying.csv  
  inflating: /content/f1_datasets/races

# Dashboard preparation

In [5]:
dashboard_df = (results_parquet
                .select(col("race_id"),col("driver_id"),col("constructor_id"),
                        col("grid"),col("fastest_lap"), col("time").alias("race_time"), col("points"), col("position"))
                .join(races_parquet
                      .select(col("race_id"),col("circuit_id"),col("race_year"),col("name").alias("race_name"),
                              col("race_timestamp").alias("race_date"))
                      .join(circuits_parquet.select(col("circuit_id"),col("location").alias("circuit_location")),'circuit_id','left'),
                      "race_id",'left')
                .join(drivers_parquet.select(col("driver_id"),col("name").alias("driver_name"),col("number").alias("driver_number"), 
                                             col("nationality").alias("driver_nationality")), 
                      "driver_id","left")
                .join(constructors_parquet.select(col("constructor_id"),col("name").alias("team")), 
                      "constructor_id","left")
                .withColumn("created_date",current_timestamp())
                )

dashboard_ddf = dashboard_df.drop("constructor_id",'driver_id',"race_id","circuit_id")
dashboard_ddf.write.mode("overwrite").parquet("/content/f1_datasets/presentation/dashboard")
dashboard_parquet = spark.read.parquet("/content/f1_datasets/presentation/dashboard")
dashboard_parquet.show()

+----+-----------+-----------+------+--------+---------+-----------------+-------------------+----------------+-----------------+-------------+------------------+-----------+--------------------+
|grid|fastest_lap|  race_time|points|position|race_year|        race_name|          race_date|circuit_location|      driver_name|driver_number|driver_nationality|       team|        created_date|
+----+-----------+-----------+------+--------+---------+-----------------+-------------------+----------------+-----------------+-------------+------------------+-----------+--------------------+
|   1|         60|1:31:10.744|  25.0|       1|     2012|Indian Grand Prix|2012-10-28 09:30:00|   Uttar Pradesh| Sebastian Vettel|            5|            German|   Red Bull|2022-11-09 15:50:...|
|   5|         60|     +9.437|  18.0|       2|     2012|Indian Grand Prix|2012-10-28 09:30:00|   Uttar Pradesh|  Fernando Alonso|           14|           Spanish|    Ferrari|2022-11-09 15:50:...|
|   2|         59|  

In [39]:
driverRankSpec = Window.partitionBy("race_year").orderBy(desc("Points"),asc("wins"))

drivers_standings_df = (dashboard_parquet
                        .groupBy('race_year','driver_name','team')
                        .agg(sum("points").alias("Points"),
                             count(when(col("position")==1,True)).alias("wins"))
                        .withColumn("rank",rank().over(driverRankSpec))
                        .orderBy(desc("race_year"),desc("Points"))
                        )
drivers_standings_df.write.mode("overwrite").parquet("/content/f1_datasets/presentation/dashboard")
drivers_standings_parquet = spark.read.parquet("/content/f1_datasets/presentation/dashboard")
drivers_standings_parquet.show()

+---------+----------------+--------------+------+----+----+
|race_year|     driver_name|          team|Points|wins|rank|
+---------+----------------+--------------+------+----+----+
|     2022|  Max Verstappen|      Red Bull| 400.0|  14|   1|
|     2022|    Sergio Pérez|      Red Bull| 270.0|   2|   2|
|     2022| Charles Leclerc|       Ferrari| 261.0|   3|   3|
|     2022|  George Russell|      Mercedes| 226.0|   0|   4|
|     2022|  Lewis Hamilton|      Mercedes| 215.0|   0|   5|
|     2022|    Carlos Sainz|       Ferrari| 201.0|   1|   6|
|     2022|    Lando Norris|       McLaren| 107.0|   0|   7|
|     2022|    Esteban Ocon|Alpine F1 Team|  79.0|   0|   8|
|     2022| Fernando Alonso|Alpine F1 Team|  71.0|   0|   9|
|     2022| Valtteri Bottas|    Alfa Romeo|  45.0|   0|  10|
|     2022|Sebastian Vettel|  Aston Martin|  36.0|   0|  11|
|     2022|Daniel Ricciardo|       McLaren|  32.0|   0|  12|
|     2022|    Pierre Gasly|    AlphaTauri|  23.0|   0|  13|
|     2022| Kevin Magnus

In [40]:
constructorsRankSpec = Window.partitionBy("race_year").orderBy(desc("Points"),asc("wins"))

constructors_standings_df = (dashboard_parquet
                             .groupBy('race_year','team')
                             .agg(sum("points").alias("Points"),
                                  count(when(col("position")==1,True)).alias("wins"))
                             .withColumn("rank",rank().over(constructorsRankSpec))
                             .orderBy(desc("race_year"),desc("Points"))
                             )
constructors_standings_df.write.mode("overwrite").parquet("/content/f1_datasets/presentation/dashboard")
constructors_standings_parquet = spark.read.parquet("/content/f1_datasets/presentation/dashboard")
constructors_standings_parquet.show()

+---------+--------------+------+----+----+
|race_year|          team|Points|wins|rank|
+---------+--------------+------+----+----+
|     2022|      Red Bull| 670.0|  16|   1|
|     2022|       Ferrari| 462.0|   4|   2|
|     2022|      Mercedes| 441.0|   0|   3|
|     2022|Alpine F1 Team| 150.0|   0|   4|
|     2022|       McLaren| 139.0|   0|   5|
|     2022|    Alfa Romeo|  51.0|   0|   6|
|     2022|  Aston Martin|  49.0|   0|   7|
|     2022|    AlphaTauri|  35.0|   0|   8|
|     2022|  Haas F1 Team|  33.0|   0|   9|
|     2022|      Williams|   8.0|   0|  10|
|     2021|      Mercedes| 604.5|   9|   1|
|     2021|      Red Bull| 578.5|  11|   2|
|     2021|       Ferrari| 322.5|   0|   3|
|     2021|       McLaren| 274.0|   1|   4|
|     2021|Alpine F1 Team| 155.0|   1|   5|
|     2021|    AlphaTauri| 142.0|   0|   6|
|     2021|  Aston Martin|  77.0|   0|   7|
|     2021|      Williams|  23.0|   0|   8|
|     2021|    Alfa Romeo|  13.0|   0|   9|
|     2021|  Haas F1 Team|   0.0